In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split  
from sklearn import preprocessing
import optuna 
import lightgbm as lgb
from sklearn.metrics import accuracy_score, roc_auc_score,precision_score, confusion_matrix,recall_score
from sklearn.model_selection import cross_val_score, train_test_split
import os 
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier

In [2]:
train_data=pd.read_csv('/home/iai/son/lastcheck/spambase_train_data.csv')
test_data=pd.read_csv('/home/iai/son/lastcheck/spambase_test_data.csv')

In [5]:
train_stage_features=['0.7', '278', '61', '0', '0.34', '0.30', '0.5', '0.44', '0.40', '0.778',
                        '0.24', '1.93', '0.39', '1.29', '0.43', '0.2', '0.36', '0.17', '0.20', 
                        '0.28', '0.21', '0.32.1', '0.41', '0.29', '3.756', '0.35', '0.25', '0.15', 
                        '0.64.1', '0.38', '0.45', '0.6', '0.33', '0.42', '0.14', '0.9', '0.10', 
                        '0.3', '0.13', '0.8', '0.19', '0.23', '0.18', '0.22', '0.16', '0.64.2', 
                        '0.1', '0.31', '0.27', '0.11', '0.4', '0.12', '0.26', '0.64', '0.37', '0.32', '0.96']
test_stage_features=['0.7', '278', '61', '0', '0.34', '0.30', '0.5', '0.44', '0.40', '0.778',
                        '0.24', '1.93', '0.39', '1.29', '0.43', '0.2', '0.36', '0.17', '0.20']

In [6]:
train_X=train_data.loc[:,train_stage_features]
train_y=train_data['1']
new_test_data_X=test_data.loc[:,test_stage_features]
new_test_data_y=test_data['1']

In [7]:
ratio=train_y.value_counts()[0]/train_y.value_counts()[1]

In [8]:
onlytrain_X=train_X.loc[:,['0.28', '0.21', '0.32.1', '0.41', '0.29', '3.756', '0.35', '0.25', '0.15', 
                        '0.64.1', '0.38', '0.45', '0.6', '0.33', '0.42', '0.14', '0.9', '0.10', 
                        '0.3', '0.13', '0.8', '0.19', '0.23', '0.18', '0.22', '0.16', '0.64.2', 
                        '0.1', '0.31', '0.27', '0.11', '0.4', '0.12', '0.26', '0.64', '0.37', '0.32', '0.96']]

In [9]:

onlytrain_X_scaled=StandardScaler().fit_transform(onlytrain_X)

In [10]:
from sklearn.decomposition import PCA 
pca=PCA(n_components=1)

In [11]:
pca.fit(onlytrain_X_scaled)

PCA(n_components=1)

In [12]:
onlytrain_X_scaled_pca=pca.transform(onlytrain_X_scaled)

In [13]:
pca_columns=['pca_component1']
onlytrain_pca=pd.DataFrame(onlytrain_X_scaled_pca,columns=pca_columns)

In [14]:
onlytrain_pca

,pca_component1
0,-0.455646
1,-0.611066
2,-0.495949
3,0.305310
4,-0.687510
...,...
3675,-0.788374
3676,-0.647861
3677,-0.643166
3678,-0.297409


In [15]:
train_X=train_data[test_stage_features] 

In [16]:
new_train_X=pd.concat([train_X,onlytrain_pca],axis=1)

In [17]:
import optuna
from lightgbm import LGBMRegressor

# Objective 함수 정의
def objective(trial):
    
    # 하이퍼파라미터 탐색할 공간 정의
    params = {
        'n_estimators': trial.suggest_int('n_estimators',100,1000),
        'max_depth': trial.suggest_int('max_depth',3,9),
        'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
    }
    
    # LGBMRegressor 모델 객체 생성
    model = LGBMRegressor(**params, random_state=42)
    
    # 교차검증 수행하여 모델 성능 측정
    scores = -1 * cross_val_score(model, train_X, onlytrain_pca['pca_component1'],
                                  cv=5, scoring='neg_mean_squared_error')
    
    # 교차검증 평균 점수 리턴
    return np.mean(scores)


# Optuna study 생성
study = optuna.create_study(direction='minimize')

# study 실행 (n_trials는 시도 횟수)
study.optimize(objective, n_trials=50)

# 최적화된 하이퍼파라미터 값 출력
print(study.best_params)

[I 2023-04-26 15:37:52,081] A new study created in memory with name: no-name-070d19a0-46ed-4986-a89b-bb167ba6a493
/tmp/ipykernel_1386719/772960702.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-26 15:37:53,533] Trial 0 finished with value: 0.8410906626429411 and parameters: {'n_estimators': 794, 'max_depth': 8, 'learning_rate': 0.024683128163763283}. Best is trial 0 with value: 0.8410906626429411.
/tmp/ipykernel_1386719/772960702.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04

{'n_estimators': 814, 'max_depth': 5, 'learning_rate': 0.0992034396407071}


## **Test data 새로운 컬럼 생성하기**

In [18]:
best_lgbmreg=LGBMRegressor(learning_rate= 0.0992034396407071,max_depth=5,n_estimators=814,random_state=42)
best_lgbmreg.fit(train_X, onlytrain_pca['pca_component1'])
new_test_data_X['pca_component1']=best_lgbmreg.predict(new_test_data_X[test_stage_features])

In [19]:
def objective(trial):
    # Define hyperparameters to optimize 
    params={
        'boosting_type':'gbdt',
        'objective':'binary',
        'metric':'binary_logloss',
        'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
        'max_depth':trial.suggest_int('max_depth',3,9),
        'n_estimators':trial.suggest_int("n_estimators",100,1000)
    }
    # Train and evaluate model 
    lgb_cv=lgb.LGBMClassifier(**params, random_state=42,scale_pos_weight=ratio)
    scores=cross_val_score(lgb_cv,new_train_X,train_y,cv=5,scoring='roc_auc')
    auc=scores.mean()
    return auc 

# Define study object and optimize 

study=optuna.create_study(direction='maximize',study_name='lgb_boost_opt',load_if_exists=True)
study.optimize(objective, n_trials=50)

# Print best hyperparameters and auc
print(f'Best hyperparameters: {study.best_params}')
print(f'Best AUC: {study.best_value:.4f}')

[I 2023-04-26 15:39:16,215] A new study created in memory with name: lgb_boost_opt
/tmp/ipykernel_1386719/14971935.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-26 15:39:17,053] Trial 0 finished with value: 0.9702310689655655 and parameters: {'learning_rate': 0.011436321684367761, 'max_depth': 9, 'n_estimators': 242}. Best is trial 0 with value: 0.9702310689655655.
/tmp/ipykernel_1386719/14971935.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-26 15:39:18,210] Trial 1 finished wi

Best hyperparameters: {'learning_rate': 0.044340301437025484, 'max_depth': 7, 'n_estimators': 720}
Best AUC: 0.9770


In [20]:

model_lgbm=LGBMClassifier(learning_rate= 0.044340301437025484, max_depth= 7, n_estimators= 720)
model_lgbm.fit(new_train_X,train_y)

LGBMClassifier(learning_rate=0.044340301437025484, max_depth=7,
               n_estimators=720)

In [21]:
newtest_pred=model_lgbm.predict_proba(new_test_data_X)[:,1]

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

def get_clf_prob(y_test, probability):
  pred=np.where(probability > 0.50,1,0)
  confusion=confusion_matrix(y_test, pred)
  accuracy=accuracy_score(y_test,pred)
  precision=precision_score(y_test,pred) 
  recall=recall_score(y_test,pred) 
  # F1 스코어 추가 
  f1=f1_score(y_test,pred,average='macro')
  Roc_score=roc_auc_score(y_test,probability)
  print('임계값: ', 0.5) 
  print('오차행렬')
  print(confusion) 
  # f1 score print 추가 
  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3:.4f}, AUC:{4: .4f}'.format(accuracy,precision,recall,f1,Roc_score))

In [23]:
get_clf_prob(new_test_data_y,newtest_pred)

임계값:  0.5
오차행렬
[[508  22]
 [ 86 304]]
정확도: 0.8826, 정밀도: 0.9325, 재현율: 0.7795, F1:0.8765, AUC: 0.9497
